# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [ ]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

: 

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [ ]:
def exp_return(data, P):
    for i in range(len(data)):
        pr = data.at[i, 'Price in 1 Year'] / P - 1
    return pr * data.at[i, 'Probability']
    # exp_return(data, 40) should output 0.07500000000000002
    return 0

: 

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [ ]:
def volatility(data, P):
    v = 0
    for i in range(len(data)):
        v += ((data.at[i, 'Price in 1 Year'] / P - 1) ** 2) * data.at[i, 'Probability']
    return (v - exp_return(data, P) ** 2) ** 0.5
    # volatility(data,40) should output 0.21301408404140795

: 

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [ ]:
def interval(data, P):
    er = exp_return(data, P)
    s_err = volatility(data, P) / (len(data) ** (1/2))
    z = 1.96
    up = er + s_err * z
    low = er - s_err * z
    return [low,up]
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]

: 

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [ ]:
def exp_return_capm(beta, market_return, rf):
    return rf + beta * (market_return - rf)
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028

: 

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [ ]:
def clean_data(path):
    data = pd.read_csv(path)
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    data.dropna(inplace=True)
    val = data['CUSIP'].value_counts()
    threshold = 500
    data = data[data['CUSIP'].map(val >= threshold)]
    # This should yield data with 403316 rows × 4 columns
    return data


: 

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [ ]:
def avg_return(data):
    group = data.groupby(['CUSIP'])
    meandf = data.groupby(['CUSIP'])['RET'].mean()
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : list(group.groups.keys()),
        "Average Return" : meandf
    })
    
    return output

: 

In [ ]:
def security_volatilities(data):
    group = data.groupby(['CUSIP'])
    stdf = data.groupby(['CUSIP'])['RET'].std()
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : list(group.groups.keys()),
        "Volatility" : stdf
    })
    
    return output

: 

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [ ]:
def correlation(ret_data, vol_data):
    ret_data.reset_index(drop=True, inplace=True)
    vol_data.reset_index(drop=True, inplace=True)
    new_df = ret_data.merge(vol_data, how='inner', on='CUSIP')
    # correlation(rets, vols) should yield 0.44837090727059614
    
    return new_df.corr()['Average Return']['Volatility']

: 